In [1]:
# ==========================================
# 🔹 1. CARGA DE LIBRERÍAS
# ==========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# ==========================================
# 🔹 2. CARGA DE DATASET Y SCHEMA
# ==========================================
schema = pd.read_csv("data/raw/survey_results_schema.csv")
df = pd.read_csv("data/raw/survey_results_public.csv")

print("Filas:", df.shape[0], " Columnas:", df.shape[1])
print("\nPrimeras columnas:\n", df.columns.tolist()[:15])

def map_ai_usage(value):
    if pd.isna(value):
        return np.nan
    v = str(value).lower()
    if v.startswith("yes"):
        return 1
    elif v.startswith("no"):
        return 0
    else:
        return np.nan

df["AI_Usage"] = df["AISelect"].apply(map_ai_usage)
df["AI_Usage"].value_counts(dropna=False)



Filas: 49123  Columnas: 170

Primeras columnas:
 ['ResponseId', 'MainBranch', 'Age', 'EdLevel', 'Employment', 'EmploymentAddl', 'WorkExp', 'LearnCodeChoose', 'LearnCode', 'LearnCodeAI', 'AILearnHow', 'YearsCode', 'DevType', 'OrgSize', 'ICorPM']


AI_Usage
1.0    26438
NaN    15437
0.0     7248
Name: count, dtype: int64

In [ ]:

# ==========================================
# 🔹 3. INSPECCIÓN DE LA VARIABLE OBJETIVO
# ==========================================
if "AI_Usage" not in df.columns:
    raise KeyError("No se encontró la columna 'AI_Usage' en el dataset.")

print("\nValores únicos en AI_Usage:", df["AI_Usage"].unique())
print("\nDistribución:")
print(df["AI_Usage"].value_counts(normalize=True))

# Eliminamos filas sin respuesta
df = df.dropna(subset=["AI_Usage"])

# ==========================================
# 🔹 4. SELECCIÓN DE VARIABLES PREDICTORAS
# ==========================================
# Seleccionamos algunas variables representativas del survey
predictors = [
    "Country",
    "EdLevel",                # Nivel educativo
    "DevType",                # Tipo de desarrollador
    "LanguageHaveWorkedWith", # Lenguajes usados
    "Industry",               # Industria
    "OrgSize",                # Tamaño de empresa
    "Age",                    # Edad (si está disponible)
    "Employment",             # Tipo de empleo
]

available_predictors = [col for col in predictors if col in df.columns]
print("\nVariables disponibles:", available_predictors)

X = df[available_predictors]
y = df["AI_Usage"]

# ==========================================
# 🔹 5. LIMPIEZA BÁSICA
# ==========================================
# Convertimos NaN a 'Unknown' para las categóricas
X = X.fillna("Unknown")

# ==========================================
# 🔹 6. DIVISIÓN TRAIN / TEST
# ==========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# ==========================================
# 🔹 7. PIPELINE DE PREPROCESAMIENTO + MODELO
# ==========================================
cat_cols = X_train.columns  # todas son categóricas en este caso

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

# ==========================================
# 🔹 8. ENTRENAMIENTO
# ==========================================
model.fit(X_train, y_train)

# ==========================================
# 🔹 9. EVALUACIÓN
# ==========================================
y_pred = model.predict(X_test)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Matriz de confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

# ==========================================
# 🔹 10. MATRIZ DE CORRELACIÓN (entre variables codificadas)
# ==========================================
# Solo para una vista exploratoria simple (opcional)
# Esto usa las versiones codificadas del conjunto de entrenamiento

X_encoded = pd.DataFrame(
    model.named_steps["preprocessor"].transform(X_train).toarray(),
    columns=model.named_steps["preprocessor"].get_feature_names_out()
)
corr = pd.DataFrame(np.corrcoef(X_encoded.T, y_train), 
                    index=X_encoded.columns.tolist()+["AI_Usage"],
                    columns=X_encoded.columns.tolist()+["AI_Usage"])

# Visualizamos solo las 10 variables con mayor correlación absoluta con AI_Usage
cor_with_ai = corr["AI_Usage"].drop("AI_Usage").abs().sort_values(ascending=False).head(10)
plt.figure(figsize=(8,5))
cor_with_ai.plot(kind='barh', color='salmon')
plt.title("Variables más correlacionadas con AI_Usage")
plt.xlabel("Correlación (absoluta)")
plt.gca().invert_yaxis()
plt.show()
